# buy-and-hold

buy, then never ever sell, until the end date :)

In [2]:
import pandas as pd
import datetime
import pinkfish as pf

# format price data
pd.options.display.float_format = '{:0.2f}'.format

Some global data

In [14]:
symbol = 'FXE'
capital = 10000
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime.now()

Timeseries

In [15]:
# fetch timeseries, select, finalize
ts = pf.fetch_timeseries(symbol)
ts = pf.select_tradeperiod(ts, start, end, use_adj=False)
ts, start = pf.finalize_timeseries(ts, start)

# create tradelog and daily balance objects
tlog = pf.TradeLog(symbol)
dbal = pf.DailyBal()

In [16]:
ts

,high,low,open,close,volume,adj_close
date,,,,,,
2015-01-02,118.46,118.16,118.42,118.20,497700.00,118.20
2015-01-05,117.66,117.26,117.27,117.54,698700.00,117.54
2015-01-06,117.73,117.05,117.09,117.16,432900.00,117.16
2015-01-07,116.74,116.19,116.37,116.49,692500.00,116.49
2015-01-08,116.36,115.91,116.05,116.08,821600.00,116.08
...,...,...,...,...,...,...
2020-09-18,111.75,111.36,111.51,111.57,100200.00,111.57
2020-09-21,110.86,110.47,110.84,110.78,105300.00,110.78
2020-09-22,110.72,110.09,110.68,110.26,132300.00,110.26


Algorithm

In [6]:
pf.TradeLog.cash = capital

# loop through timeseries
for i, row in enumerate(ts.itertuples()):

    date = row.Index.to_pydatetime()
    end_flag = pf.is_last_row(ts, i)

    # buy
    if tlog.shares == 0:
        tlog.buy(date, row.close)
    # sell
    elif end_flag:
        tlog.sell(date, row.close)

    # record daily balance
    dbal.append(date, row.high, row.low, row.close)

Retrieve logs

In [7]:
tlog = tlog.get_log()
dbal = dbal.get_log(tlog)

In [8]:
tlog.tail()

,entry_date,entry_price,exit_date,exit_price,pl_points,pl_cash,qty,cumul_total,direction,symbol
0,2015-01-02,15.11,2020-08-31,26.32,11.21,7409.21,661,7409.21,LONG,SLV


Get stats

In [9]:
stats = pf.stats(ts, tlog, dbal, capital)

Summary

In [10]:
pf.summary(stats)

,strategy
annual_return_rate,10.29
max_closed_out_drawdown,-42.77
best_month,54.60
worst_month,-34.90
sharpe_ratio,0.52
sortino_ratio,0.68
monthly_std,7.91
annual_std,16.15
